n this example, we loaded the pre-trained VGG16 model without the classification layers and froze the layers in the pre-trained model. Then we added a few fully connected layers on top of the pre-trained model for classification. We trained the model with data augmentation and evaluated it on the testing set.

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load the pre-trained VGG16 model without the classification layers
vgg = VGG16(include_top=False, input_shape=(width, height, 3))

# Freeze the layers in the pre-trained model
for layer in vgg.layers:
    layer.trainable = False

# Add the classification layers on top of the pre-trained model
x = Flatten()(vgg.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)

# Create the new model
model = Model(inputs=vgg.input, outputs=x)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])

# Early stopping to prevent overfitting
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with data augmentation
history = model.fit(datagen.flow(train_X, train_Y, batch_size=batch_size),
                    epochs=50, 
                    validation_data=(val_X, val_Y),
                    steps_per_epoch=len(train_X) // batch_size,
                    callbacks=[es])
                    
# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)
